# Notebook 12: Working with Physical Units

Underworld3 has built-in support for physical units throughout the modeling workflow. This makes your models easier to understand and helps catch dimensional errors early.

In this notebook you'll learn:
- Creating physical quantities (temperatures, velocities, viscosities)
- Converting between units
- Working with unit-aware arrays and coordinates
- Automatic unit tracking through derivatives

In [1]:
import nest_asyncio
nest_asyncio.apply()

import underworld3 as uw
import numpy as np
import sympy

## Creating Physical Quantities

Create quantities using `uw.units`, which provides access to the Pint units library. You can write units explicitly or use strings:

In [2]:
# Explicit units
plate_velocity = 5 * uw.units.cm / uw.units.year
mantle_temperature = 1500 * uw.units.K

# String notation (Pint can parse these)
mantle_depth = 2900 * uw.units("km")
mantle_viscosity = 1e21 * uw.units("Pa.s")

# Display (implicit - last line shows result)
mantle_temperature

<Quantity(1500, 'kelvin')>

In [3]:
# View all the quantities
print(f"Depth:       {mantle_depth}")
print(f"Velocity:    {plate_velocity}")
print(f"Temperature: {mantle_temperature}")
print(f"Viscosity:   {mantle_viscosity}")

Depth:       2900 kilometer
Velocity:    5.0 centimeter / year
Temperature: 1500 kelvin
Viscosity:   1e+21 pascal * second


### Unit Conversions

Converting between compatible units is straightforward using the `.to()` method:

In [4]:
# Convert velocity
plate_velocity.to("mm/year")

<Quantity(50.0, 'millimeter / year')>

In [5]:
# Convert depth
mantle_depth.to("m")

<Quantity(2900000.0, 'meter')>

In [6]:
# Display the units registry for reference
uw.units.view()

## Units Registry

The Underworld3 units registry provides access to physical units for dimensional analysis.

### Common Units Examples:
- **Temperature**: `uw.units.K`, `uw.units.celsius`, `uw.units.degC`
- **Pressure**: `uw.units.Pa`, `uw.units.bar`, `uw.units.atm`
- **Length**: `uw.units.m`, `uw.units.cm`, `uw.units.km`
- **Time**: `uw.units.s`, `uw.units.year`, `uw.units.Ma` (million years)
- **Viscosity**: `uw.units.Pa * uw.units.s`
- **Velocity**: `uw.units.cm / uw.units.year`

### Usage:
```python
# Create quantities
temperature = 1500 * uw.units.K
viscosity = 1e21 * uw.units.Pa * uw.units.s
velocity = 5 * uw.units.cm / uw.units.year

# Set model reference quantities
model.set_reference_quantities(
    mantle_temperature=temperature,
    mantle_viscosity=viscosity
)
```

**Total units available**: 1086


## Working with Mesh Coordinates

Create a simple mesh and variables to explore unit-aware operations:

In [7]:
# Create a mesh
mesh = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0.0, 0.0),
    maxCoords=(1000.0, 500.0),
    cellSize=50.0,
    qdegree=2,
)

print(f"Mesh has {mesh.X.coords.shape[0]} nodes")

Mesh has 274 nodes


In [8]:
# Create mesh variables with units
temperature = uw.discretisation.MeshVariable("T", mesh, 1, degree=2, units="K")
velocity = uw.discretisation.MeshVariable("u", mesh, 2, degree=2, units="m/s")

### Initializing Fields

Set up a simple temperature field:

In [9]:
# Get coordinate symbols
x, y = mesh.X

# Initialize temperature: T = 300 + 2.6*y (K)
# Use evaluate for proper initialization across all degrees of freedom
with uw.synchronised_array_update():
    temperature.array[...] = uw.function.evaluate(300.0 + 2.6 * y, temperature.coords)
    velocity.array[...] = uw.function.evaluate(sympy.Matrix([5.0, 0.0]), velocity.coords).reshape(velocity.array.shape)

print(f"Temperature range: {temperature.min():.1f} to {temperature.max():.1f}")
print(f"Variable units: {temperature.units}")

Temperature range: 300.0 to 1600.0
Variable units: kelvin


## Unit-Aware Operations

Variables with units support mathematical operations that preserve dimensional consistency:

In [10]:
# Get units from expressions
uw.get_units(temperature)

<Unit('kelvin')>

In [11]:
# Check dimensionality
uw.get_dimensionality(temperature)

<UnitsContainer({'[temperature]': 1})>

In [12]:
# Units work naturally with arithmetic
uw.get_units(temperature / velocity[0])

<Unit('second * kelvin / meter')>

### Automatic Unit Tracking Through Derivatives

When you take derivatives, units are tracked automatically:

In [13]:
# Derivative automatically has correct units
dTdy = temperature.diff(y)[0]

# The derivative has units! 
uw.get_units(dTdy)

<Unit('kelvin')>

The derivative has units that make physical sense - it knows about both the temperature units and the coordinate system.

## Computing Gradients with Projection

To get numerical gradient values at mesh nodes, use the Projection system:

In [14]:
# Create a variable to hold the gradient  
gradT = uw.discretisation.MeshVariable(
    "gradT", 
    mesh, 
    1, 
    degree=1, 
    units="K/m"  # Specify units directly
)

# Project the derivative onto the mesh
# The projection solver now automatically unwraps symbolic objects
proj = uw.systems.Projection(mesh, gradT)
proj.uw_function = temperature.diff(y)[0]  # No .sym needed!
proj.solve()

print(f"Gradient units: {gradT.units}")
print(f"Gradient range: {gradT.min():.3f} to {gradT.max():.3f}")
# print(f"Gradient range: {gradT.global_min():.3f} to {gradT.global_max():.3f}")

Gradient units: kelvin / meter
Gradient range: 2.600 to 2.600


In [15]:
gradT.array[100,...]

UnitAwareArray([[2.6]]), callbacks=0, units='kelvin / meter')

### Converting Gradient Units

In [16]:
# Get the gradient array (unit-aware)
gradT_array = gradT.array

# Convert to different units
gradT_km = gradT_array.to("K/km")

print(f"Gradient in K/m:  {gradT_array[0, 0, 0]:.6f}")
print(f"Gradient in K/km: {gradT_km[0, 0, 0]:.3f}")

Gradient in K/m:  2.600000
Gradient in K/km: 2600.000


## Normalized Coordinates

Sometimes you want dimensionless coordinates. Unit division makes this natural:

In [17]:
# Get domain bounds
ymin = uw.expression(r"y_{min}", mesh.X.coords[:, 1].min(), "min y coord")
ymax = uw.expression(r"y_{max}", mesh.X.coords[:, 1].max(), "max y coord")

# Normalized y coordinate (dimensionless)
y_normalized = y / ymax

# Check it's dimensionless (should be None)
uw.get_units(y_normalized) is None

True

## Summary

The units system in Underworld3:

- **Creation**: Use `uw.units` to create physical quantities
- **Conversion**: Use `.to(target_units)` to convert between compatible units
- **Derivatives**: Automatically get correct units (e.g., `temperature.diff(y)` has units K/m)
- **Dimensionless**: Division by same units gives dimensionless results
- **Type checking**: `uw.get_units()` and `uw.get_dimensionality()` let you inspect units

Units make your code clearer and help catch errors early!

### What's Next?

**Notebook 13** introduces **reference quantities** and shows how to:
- Choose good reference units for your problem
- Set model-wide coordinate systems
- Work with meshes that have proper coordinate units
- Convert easily between different unit systems

This makes working with physical problems even more natural!

## Try It Yourself

Exercises to explore:

```python
# 1. Create different quantities
density = 3300 * uw.units("kg/m^3")
gravity = 9.81 * uw.units("m/s^2")
stress = density * gravity * mantle_depth

# 2. Check the units
uw.get_units(stress)

# 3. Convert to different units
stress.to("GPa")

# 4. Create a vector derivative
div_velocity = velocity[0].diff(x) + velocity[1].diff(y)
uw.get_units(div_velocity)
```